In [1]:
!pip install confluent-kafka

In [2]:
!pip install confluent-kafka[avro]

In [3]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [4]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [5]:
conf = {'bootstrap.servers': bootstrap_server}

In [6]:
a = AdminClient(conf)

In [7]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 11 topics:
  "__transaction_state" with 50 partition(s)
  "_kafka-connect-01-offsets" with 25 partition(s)
  "_schemas" with 1 partition(s)
  "_confluent-ksql-confluent_rmoff_01_command_topic" with 1 partition(s)
  "SmokeSensorEvent" with 1 partition(s)
  "iot-oven" with 1 partition(s)
  "_kafka-connect-01-configs" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "_kafka-connect-01-status" with 5 partition(s)
  "ratings" with 1 partition(s)
  "confluent_rmoff_01ksql_processing_log" with 1 partition(s)


In [8]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

## Define the value schema

In [9]:
value_schema_str = """
{
  "namespace": "hoven",
  "type": "record",
  "name": "TemperatureSensorEvent",
  "fields": [
    {
      "name": "sensor",
      "type": "string"
    },
    {
      "name": "temperature",
      "type": "float"
    },
    {
      "name": "ts",
      "type": "long"
    }
  ]
}
"""

In [10]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
import time

topic = "TemperatureSensorEvent"
schema_registry_url = "http://schema-registry:8081"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [11]:
schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [12]:
avro_serializer = AvroSerializer(value_schema_str, schema_registry_client)

producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': avro_serializer
}

producer = SerializingProducer(producer_conf)

In [13]:
from random import gauss
i = 0;

while True:
    i += 1
    key = "S1"
    value = {"sensor": "S1","temperature": gauss(20, 5.0),"ts":int(time.time()*1000.0)}
    producer.produce(topic=topic, value=value, key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)


{'sensor': 'S1', 'temperature': 28.2353344077577, 'ts': 1605589368191}
{'sensor': 'S1', 'temperature': 18.029587531471737, 'ts': 1605589379213}


KeyboardInterrupt: 